In [1]:
import numpy as np
from itertools import product

In [2]:
class Layer:
    def __init__(self, nodes, activation='relu', input_features=None):
        self.nodes = nodes
        self.weights = None
        self.bias = None
        self.activation = activation
        self.input_features = input_features
        self.z = None # pre-activation
        self.a = None # After activation

    def activation_function(self, x):
        if self.activation == 'relu':
            return np.maximum(0, x)

        elif self.activation == 'softmax':
            return np.exp(x) / np.sum(np.exp(x), axis=0)

    def forward(self, inputs):
        self.z = np.dot(self.weights, inputs) + self.bias
        self.a = self.activation_function(self.z)

        return self.a

In [17]:
class Model:
    def __init__(self, layers, loss='mse', epochs=10, learning_rate=0.001, metrics=False):
        self.layers = layers
        self.loss = loss
        self.lr = learning_rate
        self.epochs = epochs
        self.metrics = metrics

    def predict(self, inputs):
        y_pred = []

        for input in inputs:
            for i in range(len(self.layers)):
                if i == 0:
                    input = self.layers[i].forward(input.reshape(inputs[0].shape[0], 1))
                else:
                    input = self.layers[i].forward(input)

            y_pred.append(input[0])

        return np.array(y_pred)

    def generate_patterns(self, arr):
        constants = [item[0] for item in arr]
        ranges = [range(1, item[1] + 1) for item in arr]
        combinations = product(*ranges)

        result = []
        for pattern in combinations:
            paired = [[constants[i], pattern[i]] for i in range(len(arr))]
            result.append(paired)

        return result

    def summary(self):
        total_trainable_params = 0

        for i in range(len(self.layers)):
            trainable_weights = self.layers[i].weights.shape[0] * self.layers[i].weights.shape[1]
            trainable_bias = self.layers[i].bias.shape[0]
            trainable_params = trainable_weights + trainable_bias
            total_trainable_params += trainable_params

            print(f"trainable parameters (Layer {i+1}): {trainable_params}")
        print(f"\ntrainable parameters (Total)  : {total_trainable_params}")

    def compile(self):
        for i in range(len(self.layers)):
            layer = self.layers[i]

            if i == 0:
                layer.weights = np.ones((layer.nodes, layer.input_features))
                layer.bias = np.zeros((layer.nodes, 1))

            else:
                layer.weights = np.ones((layer.nodes, self.layers[i-1].nodes))
                layer.bias = np.zeros((layer.nodes, 1))

    def fit(self, x_train, y_train):
        for epoch_iteration in range(1, self.epochs + 1): # No of epochs
            for xi, yi in zip(x_train, y_train): # For each xi, yi
                yi_pred = self.predict([xi])
                loss = yi[0] - yi_pred[0][0]

                for layer_i in range(len(self.layers), 0, -1): # i is ith layer, start from last
                    n, m = self.layers[layer_i - 1].weights.shape

                    for j in range(1, n+1):
                        for k in range(1, m+1): # weight update
                            ways_out = self.generate_patterns([[i, self.layers[i-1].bias.shape[0]] for i in range(layer_i + 1, len(self.layers) + 1)])
                            w_net = 0

                            for p in ways_out:
                                w = 0 if len(p) == 0 else 1

                                for o in range(len(p)):
                                    w *= self.layers[p[o][0] - 1].weights[p[o][1] - 1][(j-1) if o == 0 else (p[o-1][1]-1)]
                                w_net += w

                            w_net = 1 if w_net == 0 else w_net
                            delta = -2 * loss * self.lr * w_net / len(x_train)

                            if (k == 1): # update bias
                                self.layers[layer_i - 1].bias[j - 1][0] -= delta

                            if (layer_i - 1) - 1 < 0: delta *= xi[k - 1]
                            else: delta *= self.layers[(layer_i - 1) - 1].a[k - 1][0]

                            self.layers[layer_i - 1].weights[j-1][k-1] -= delta # Update Weight

            if self.metrics:
                print(f"epoch: {epoch_iteration}, error: {np.mean(model.predict(x_train) - y_train)}")

        if self.metrics == False:
            print(f"error: {np.abs(np.mean(model.predict(x_train) - y_train))}")

In [27]:
layer_1 = Layer(input_features=2, nodes=3)
layer_3 = Layer(nodes=2)
layer_2 = Layer(nodes=1)

x_train = np.array([[1, 2], [2, 4], [3, 6], [4, 8], [5, 10]])
y_train = np.array([[10], [20], [30], [40], [50]])

model = Model(layers=[layer_1, layer_3, layer_2], epochs=10)

model.compile()
model.summary()

model.fit(x_train, y_train)
model.predict(x_train)

trainable parameters (Layer 1): 9
trainable parameters (Layer 2): 8
trainable parameters (Layer 3): 3

trainable parameters (Total)  : 20
error: 0.06063955486060237


array([[ 9.8958134 ],
       [19.91758692],
       [29.93936045],
       [39.96113397],
       [49.98290749]])